In [1]:
from __future__ import print_function

import subprocess
import seaborn as sns
import pandas as pd
import DyMat

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import math
from tqdm import tqdm
import os

import tempfile

In [11]:
%cd seesaw3_build/

/home/lukas/Projects/TrussFab/src/system_simulation/seesaw3_build


In [21]:
overrides = "startTime=0,stopTime=10,stepSize=0.2,outputFormat=mat,variableFilter=revLeft.*"

In [5]:
time subprocess.call(["./seesaw3", "-override springDamperParallel1.c='10"])

CPU times: user 705 µs, sys: 7.61 ms, total: 8.31 ms
Wall time: 26.6 ms


0

In [22]:
subprocess.run("./seesaw3 -outputPath={} -override={}".format('.', overrides, 100), shell=True, capture_output=True)

CompletedProcess(args='./seesaw3 -outputPath=. -override=startTime=0,stopTime=10,stepSize=0.2,outputFormat=mat,variableFilter=revLeft.*', returncode=0, stdout=b'LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n', stderr=b'')

In [ ]:
def getFreq(c=100):
    with tempfile.TemporaryDirectory(dir='.') as tempdir:
        subprocess.run("./seesaw3 -outputPath={} -override springDamperParallel1.c='{}'".format(tempdir, c), check=True, shell=True)
        d = DyMat.DyMatFile(tempdir + '/seesaw3_res.mat')
    return d['revLeft.phi']

def getFreqPlot(c=100):
    return pd.DataFrame(getFreq(c)).plot(title="C={}".format(c))

In [ ]:
def range_metric(array: np.array):
    return array.ptp()

In [ ]:
time getFreq(10)

In [ ]:
interact(getFreqPlot, c=(0,20000,100))

In [ ]:
time getFreq(100)

In [ ]:
ranges = []

for c in tqdm(range(0, 20000, 100)):
    ranges.append(range_metric(getFreq(c)))
    
pd.DataFrame(ranges).plot()

In [ ]:
import time
import concurrent.futures

e = concurrent.futures.ThreadPoolExecutor(4)
s = range(0, 20000, 100)
calc = lambda c: range_metric(getFreq(c))

res = [x for x in tqdm(e.map(calc, s))]

In [ ]:
pd.DataFrame(res).plot()